# Langchain and RAG Testing

Initialize the model

In [13]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

Asking something in langchain

In [14]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand what love is.")
]

In [15]:
res = chat(messages)
print(res.content)

Love is a complex and multifaceted emotion that can manifest in many different ways. It involves feelings of deep affection, care, and connection towards someone or something. Love can be romantic, platonic, familial, or even love for hobbies or interests. It often involves a combination of physical attraction, emotional attachment, and a desire to support and nurture the other person. Love can bring joy, fulfillment, and a sense of belonging to our lives.


To maintain context, you need to append extra messages to the previous chain

In [16]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Do you believe in love?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

As an AI, I don't have personal beliefs or feelings. However, I can provide information and insights on the topic based on the data and knowledge available to me. Love is a universal human experience that has been studied and explored in various fields, including psychology, sociology, and philosophy. It plays a significant role in human relationships and well-being.


GPT 3.5 is not supposed to have hallucinations, I'm going to ask it something it knows nothing about to test this

In [17]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What do you know about my classmate Ismael Picazo?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [18]:
print(res.content)

I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me during our conversation. If you have any specific questions or topics you'd like to discuss, feel free to let me know.


Lets add some more context and tell it who Ismael is

In [19]:
llmchain_information = [
    "Ismael Picazo is a student at IE University in Madrid, great guy.",
    "Ismael is known for his work in the field of AI and machine learning.",
    "Ismael has a great future ahead of him."
]

source_knowledge = "\n".join(llmchain_information)

Now we feed it the information just like this

In [21]:
query = "Can you tell me about Ismael Picazo?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

In [22]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [23]:
print(res.content)

Based on the provided contexts, Ismael Picazo is a student at IE University in Madrid who is known for his work in the field of AI and machine learning. He is described as a great guy with a promising future ahead of him.
